In [182]:
import pandas as pd
import numpy as np
from copy import deepcopy # Add Deepcopy for args

import seaborn as sns 
import matplotlib.pyplot as plt
import os, pickle, joblib, argparse

from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from statsmodels.tsa.api import SimpleExpSmoothing

# pytorch
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
 
 
print(torch.__version__)
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (8, 6)

1.6.0+cpu
Populating the interactive namespace from numpy and matplotlib


C:\Anaconda3\envs\venv\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['seed', 'partition']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [183]:
df = pd.read_hdf('./data/merged_allcoin.h5')

In [184]:
merged_arr = df2d_to_array3d(df)

DataFrame to array, Complete!


In [185]:
# ================================================= #
def df2d_to_array3d(df_2d):
    
    # 입력 받은 2차원 데이터 프레임을 3차원 numpy array로 변경하는 함수
    feature_size = df_2d.iloc[:,2:].shape[1]
    time_size = len(df_2d.time.value_counts())
    sample_size = len(df_2d.sample_id.value_counts())
    array_3d = df_2d.iloc[:,2:].values.reshape([sample_size, time_size, feature_size])
    
    print('DataFrame to array, Complete!')
    
    return array_3d
# ================================================= #

def train_tset_spliter(arr):
    n = len(arr)
    num_features = arr.shape[2]
    
    train_arr = arr[0:int(n*0.8), :, :]
    val_arr = arr[int(n*0.8):, :, :]
    
    train_len = len(train_arr) 
    val_len = len(val_arr)
    
    print(
    f'''
    ======================================================
    Origin length is {n}, then total split length is {train_len, val_len}
    ======================================================
    train length is {train_arr.shape},
    val length is {val_arr.shape},
    num_features is ({num_features})
    '''
    )
    return train_arr, val_arr
# ================================================= #

def kbin_discretizer(input_array):

    kb = KBinsDiscretizer(n_bins=10, strategy='uniform', encode='ordinal')
    processed_data = np.zeros((input_array.shape[0], input_array.shape[1], 1))
    
    for i in range(input_array.shape[0]):
        # coin_index_export args : (input_array, coin_num)
        globals()['processing_array{}'.format(i)] = input_array[i,:,1]
        
        #globals()['outliery_array{}'.format(i)] = train_y_array[outlier[i],:,1]
        kb.fit(globals()['processing_array{}'.format(i)].reshape(input_array.shape[1],1))
        globals()['processed_fit{}'.format(i)] = kb.transform(globals()['processing_array{}'.format(i)].reshape(input_array.shape[1],1))
        
        #globals()['outliery_fit{}'.format(i)] = kb.transform(globals()['outliery_array{}'.format(i)].reshape(120,1))
        processed_data[i,:,:] = globals()['processed_fit{}'.format(i)]
        
    return processed_data

def outlier_detecter(raw_y_arr, outlier_criteria = 0.05):

    open_arr = raw_y_arr[:, :, 1] #open col is 1

    outlier_list = []
    openrange_list = []

    for idx, temp_arr in enumerate(open_arr):
    
        temp_min = temp_arr.min()
        temp_max = temp_arr.max()
        temp_arr_range = temp_max - temp_min
        openrange_list.append(temp_arr_range)

        if temp_arr_range > outlier_criteria:
            outlier_list.append(idx)
            print(f'{idx}번째 open series is outlier sample!')
            print(f'temp array range is {temp_arr_range:.3}\n')
            

    return outlier_list, np.array(openrange_list)

In [4]:
# ================================================= #
def simple_exponetial_smoothing_fory(arr, alpha=0.3):
    
    y_series = list()

    for temp_arr in arr:
        target_series = temp_arr[:, 1].reshape(-1) # open col is 1 index

        smoother = SimpleExpSmoothing(target_series, initialization_method="heuristic").fit(smoothing_level=alpha,optimized=False)
        smoothing_series = smoother.fittedvalues
        y_series.append(smoothing_series)
            
    return np.array(y_series)

# ================================================= #
def simple_exponetial_smoothing_forX(arr, alpha=0.3):
    
    # initialization
    sample_size = int(arr.shape[0])
    time_size = int(arr.shape[1])
    feature_size = int(arr.shape[2])
    
    # create empty array
    smoothing_arr = np.zeros((sample_size, time_size, feature_size - 1))

    for idx, temp_arr in enumerate(arr):
        for col in range(1, feature_size): # open col is 1 index
            if col < 5:

                temp_series = temp_arr[:, col].reshape(-1) 
                smoother = SimpleExpSmoothing(temp_series, initialization_method="heuristic").fit(smoothing_level=0.3,optimized=False)
                temp_smoothing_series = smoother.fittedvalues
                smoothing_arr[idx, :, col-1] = temp_smoothing_series

            else:
                
                pass_series = temp_arr[:, col].reshape(-1)
                smoothing_arr[idx, :, col-1] = pass_series

    return smoothing_arr
# ================================================= #

In [5]:
def plot_series(x_series, y_series):
    #입력 series와 출력 series를 연속적으로 연결하여 시각적으로 보여주는 코드 입니다.
    plt.plot(x_series, label = 'input_series')
    plt.plot(np.arange(len(x_series), len(x_series)+len(y_series)),
             y_series, label = 'output_series')
    plt.axhline(1, c = 'red')
    plt.legend()

In [61]:
# df to array 
raw_x_arr = df2d_to_array3d(raw_x_df)
raw_y_arr = df2d_to_array3d(raw_y_df)


DataFrame to array, Complete!
DataFrame to array, Complete!


In [69]:

# train set smoothing
train_x_arr = simple_exponetial_smoothing_forX(raw_x_arr)
train_y_arr = simple_exponetial_smoothing_fory(raw_y_arr)
print('simple exponetial smoothing Complete!!!')

simple exponetial smoothing Complete!!!


In [103]:
smooth_merged_arr = simple_exponetial_smoothing_forX(merged_arr)

In [104]:
data = smooth_merged_arr[0]


In [107]:
# X_train = []
# y_train = []

# n_future = 120 # Number of days we want to predict into the future.
# n_past = 360 # Number of past days we want to use to predict future.

# for i in range(n_past, data.shape[0] - n_future + 1):
        
#         X_train.append(data[i - n_past:i, :])
#         y_train.append(data[i:i+n_future, 0])
        
# X_train, y_train = np.array(X_train), np.array(y_train)
# print('X_train shape == {}.'.format(X_train.shape))
# print('y_train shape == {}.'.format(y_train.shape))

# y_train = time_split(y_train)


In [216]:
def time_split(input_array, split_size = 6):

    # origin size define
    origin_time_size = input_array.shape[0]

    # new array size define
    new_time_size = int(origin_time_size/split_size) # 1380 / 6
    new_array = np.zeros((new_time_size))

    for time_idx in range(new_time_size):

            first_time_idx = time_idx * split_size
            new_array[time_idx] = input_array[first_time_idx] #coin_num
            
    return new_array


In [109]:
data.shape

(1500, 9)

In [255]:
# train & val set
train_data, val_data  = train_test_split(data, test_size = 0.4, shuffle=False)

print(
f'''
======================================================
train X length is {len(train_data)}, train y length is {len(val_data)},
'''
)


train X length is 900, train y length is 600,



In [270]:
# ====== initialization
parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("device is",args.device)

seed = 777
np.random.seed(seed)
torch.manual_seed(seed)


# ====== Model Capacity options ===== #
args.input_dim = 9
args.hidden_dim = 100
args.output_dim = 20
args.n_layers = 1
args.batch_size = 32
args.dropout = 0.1
args.use_bn = True


# ====== Dataset Generating options ====== #
args.x_frames = 360

# ====== Model training options ===== #
args.num_epoch = 50
args.learning_rate = 0.0001
args.L2_rate = 0.00001


device is cpu


In [283]:
class WindowGenerator(Dataset):
    ''' Dataset Generate'''
    def __init__(self, data, x_frames, y_frames):
    
        self.data = data
        self.x_frames = x_frames
        self.y_frames = y_frames
    
    def __len__(self):
        return (len(self.data) - (self.x_frames + self.y_frames) + 1)

    def __getitem__(self, idx):

        idx = idx + self.x_frames
        temp_data = self.data[idx-self.x_frames : idx+self.y_frames]

        temp_data = np.log(temp_data + 1) - np.log(temp_data[self.x_frames-1, :] + 1) 
        
        X = temp_data[:self.x_frames]
        y = temp_data[self.x_frames:, 0]
        y = time_split(y, 6).argmax()

        return X, y
    
    def time_split(self, input_array, split_size = 6):

        # origin size define
        origin_time_size = input_array.shape[0]

        # new array size define
        new_time_size = int(origin_time_size/split_size) # 1380 / 6
        new_array = np.zeros((new_time_size))

        for time_idx in range(new_time_size):

                first_time_idx = time_idx * split_size
                new_array[time_idx] = input_array[first_time_idx] #coin_num

        return new_array



In [272]:
class LSTM(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout, use_bn):
        super(LSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        self.dropout = dropout
        self.use_bn = use_bn 
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers, batch_first =True)

        self.fc = self.make_regressor()
        
    def init_hidden(self, batch_size):
        return (torch.zeros(self.num_layers, batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, batch_size, self.hidden_dim))
    
    def make_regressor(self):
        layers = []
        if self.use_bn:
            layers.append(nn.BatchNorm1d(self.hidden_dim))
        layers.append(nn.Dropout(self.dropout))
        
        layers.append(nn.Linear(self.hidden_dim, self.hidden_dim // 2))
        layers.append(nn.ReLU())
        layers.append(nn.Linear(self.hidden_dim // 2, self.output_dim))
        regressor = nn.Sequential(*layers)
        return regressor
    
    def forward(self, X):
        lstm_out, self.hidden = self.lstm(X)
        y_pred = self.fc(lstm_out[:, -1, :])
        return y_pred

In [288]:
def train(model, partition, optimizer, loss_fn, args):
    ''' model training '''
   
    # data load
    trainloader = DataLoader(partition['train'],
                             batch_size = args.batch_size,
                             shuffle = True, drop_last = True)
    
    # model's mode setting
    model.train()
    model.zero_grad()
    optimizer.zero_grad()
    
    train_loss = 0.0
    for i, (X, y) in enumerate(trainloader):
    
        X = X.float().to(args.device)
        y_true = y.long().to(args.device)
        
#         print(X.shape, y_true.shape)
        
        # zero the gradient
        model.zero_grad()
        optimizer.zero_grad()
        model.hidden = model.init_hidden(X.shape[1])

        y_pred = model(X)
#         print(y_pred.shape)

        loss = loss_fn(y_pred, y_true)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()

    train_loss = train_loss / len(trainloader)
    return model, train_loss


def validate(model, partition, loss_fn, args):
    ''' model validate '''
    
    # data load
    valloader = DataLoader(partition['val'], 
                           batch_size = args.batch_size, 
                           shuffle = False, drop_last = True)
    
    # model's mode setting
    model.eval()
    val_loss = 0.0
    correct, total = 0, 0
    
    # evaluate
    with torch.no_grad():
        for i, (X, y) in enumerate(valloader):
            
            X = X.float().to(args.device)
            y_true = y.long().to(args.device)
            
            model.hidden = model.init_hidden(X.shape[1])
            # en-decoder outputs tensor 
            y_pred = model(X)
            
            # compute the loss
            loss = loss_fn(y_pred, y_true)
            val_loss += loss.item()
            
            preds = F.log_softmax(y_pred, dim=1).argmax(dim=1)
            total += y_true.size(0)
            correct += (preds == y_true).sum().item()
            
    val_loss = val_loss / len(valloader)
    val_acc = correct / total
    return val_loss, val_acc

In [294]:
def experiment(partition, args):

    model = LSTM(args.input_dim, args.hidden_dim, args.output_dim, args.n_layers, args.dropout, args.use_bn)
    model.to(args.device)
    
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=args.learning_rate, weight_decay=args.L2_rate)
    
    # epoch-wise loss
    train_losses = []
    val_losses = []

    for epoch in range(args.num_epoch):
        
        start_time = time.time()
        model, train_loss = train(model, partition, optimizer, loss_fn, args)
        val_loss, val_acc = validate(model, partition, loss_fn, args)
        end_time = time.time()
        
        # add epoch loss
        train_losses.append(train_loss)
        val_losses.append(val_loss)

        
        print('Epoch {}, Acc(val): {:2.2f}, Loss(train/val) {:2.5f}/{:2.5f}. Took {:2.2f} sec'.format(
            epoch+1, val_acc, train_loss, val_loss, end_time-start_time))
    

    # ======= Add Result to Dictionary ======= #
    result = {}
    
    result['train_losses'] = train_losses
    result['val_losses'] = val_losses
        
    return vars(args), result, model

In [295]:
# train_X, val_X, train_y, val_y = train_test_split(raw_x_arr[:, :, 1:], y_train, test_size = 0.1, shuffle=False)
trainset = WindowGenerator(train_data, 360, 120)
valset = WindowGenerator(val_data, 360, 120)

partition = {'train': trainset, 'val':valset}

In [296]:
print(args)
setting, result, model = experiment(partition, deepcopy(args))

Namespace(L2_rate=1e-05, batch_size=32, device='cpu', dropout=0.1, hidden_dim=100, input_dim=9, learning_rate=0.0001, n_layers=1, num_epoch=50, output_dim=20, use_bn=True, x_frames=360)
Epoch 1, Acc(val): 0.04, Loss(train/val) 3.01455/2.98321. Took 6.74 sec
Epoch 2, Acc(val): 0.05, Loss(train/val) 2.99896/2.98293. Took 6.57 sec
Epoch 3, Acc(val): 0.05, Loss(train/val) 2.99550/2.98307. Took 6.61 sec
Epoch 4, Acc(val): 0.03, Loss(train/val) 2.99536/2.98233. Took 6.58 sec
Epoch 5, Acc(val): 0.07, Loss(train/val) 2.98986/2.97796. Took 6.59 sec
Epoch 6, Acc(val): 0.10, Loss(train/val) 2.97148/2.96920. Took 6.58 sec
Epoch 7, Acc(val): 0.14, Loss(train/val) 2.97009/2.95594. Took 6.61 sec
Epoch 8, Acc(val): 0.12, Loss(train/val) 2.96812/2.94753. Took 6.72 sec
Epoch 9, Acc(val): 0.14, Loss(train/val) 2.96374/2.94166. Took 6.60 sec
Epoch 10, Acc(val): 0.14, Loss(train/val) 2.95768/2.93847. Took 6.59 sec
Epoch 11, Acc(val): 0.14, Loss(train/val) 2.95359/2.93529. Took 6.63 sec
Epoch 12, Acc(val): 

In [306]:
testloader = DataLoader(partition['val'], batch_size = 1, shuffle = True, drop_last = True)
model.eval()
# evaluate

with torch.no_grad():
    for i, (X, y) in enumerate(testloader):

        X = X.float().to(args.device)
        y_true = y.long().to(args.device)
                    
        # en-decoder outputs tensor 
        y_pred = model(X)
        y_pred = F.log_softmax(y_pred, dim=1).argmax(dim=1)
            
        
        # y values to cpu
        y_true = y_true.cpu().detach().numpy()
        y_pred = y_pred.cpu().detach().numpy()
        
        print(y_true, y_pred)

#         plt.plot(y_true, label = 'True series')
#         plt.legend()
#         plt.show()
        
#         plt.plot(y_pred, 'red', label = 'Prediction series')
# #         plt.plot(.max(), '*')
#         plt.legend()
#         plt.show()
        
        print("=============================================")

        if i == 10:
            break


[0] [0]
[5] [19]
[12] [8]
[12] [0]
[17] [0]
[11] [8]
[6] [0]
[12] [0]
[13] [19]
[13] [19]
[14] [8]
